# **Project 3: Did Singapore's Baby Bonus Increase 2nd Child Births?**
## A Difference-in-Difference Analysis

## Background
In the first two projects, we looked at Singapore's low fertility rates, and the Singapore Government's response to it. In particular, in 2001, Singapore introduced the **Baby Bonus** scheme to combat declining fertility rates. Mothers were given a cash subsidy of $3,000 if they were having a second child, and $4,000 if they were having a third child. The policy was not extended to 1st children until 2004.

Whilst we mentioned the subsidy in Project 1 and looked at birth-rates in Singapore after the introduction of the policy, we were critically not able to ascertain causality. For this project, I'm going to attempt to do so using a method that I've learnt about in my undergrade studies - Difference-in-Difference (DiD).

The application of the policy for only first child from 2001-2004 creates a natural experiment of sorts. Assuming that birth trends are related, and that the populations are similar we can derive a treatment and control group:
    - **Treatment Group:** 2nd child births (received incentive starting 2001)
    - **Control Group:** 1st child births (did NOT receive incentive until 2004)

By comparing the change in 2nd births vs 1st births before and after 2001, we can estimate the **causal effect** of the policy.


## Project Prompt

* **Dataset(s) to be used:** [Live-Births By Birth Order, Annual (SingStat)](https://tablebuilder.singstat.gov.sg/) - `M810081.csv`
* **Analysis question:** Did the 2001 Baby Bonus policy in Singapore for 2nd children cause an increase in 2nd child births?
* **Columns that will (likely) be used:**
    * `Data Series` (Birth Order: 1st, 2nd, 3rd...)
    * Year columns (1997-2024)
* **Hypothesis:** The Baby Bonus for 2nd children (introduced in 2001) did not lead to an increase in 2nd child births relative to 1st child births.




In [107]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.io as pio
pio.renderers.default = "notebook_connected"

## 1. Retrive Data

We first get the data from the Singapore Government's statistics website. 

The CSV in question contains total live births by birth order (first child, second child etc)

In [108]:
df = pd.read_csv('M810081.csv', skiprows=10)
df.head()

,Data Series,2024,2023,2022,2021,2020,2019,2018,2017,2016,...,1976,1975,1974,1973,1972,1971,1970,1969,1968,1967
0,Total Live-Births By Birth Order,33703.0,33541.0,35605.0,38672.0,38590.0,39279.0,39039.0,39615.0,41251.0,...,42783.0,39948.0,43268.0,48269.0,49678.0,47088.0,45934.0,44562.0,47241.0,50560.0
1,1st Live-Birth,15769.0,15552.0,16739.0,17895.0,18414.0,18553.0,18211.0,18570.0,19392.0,...,17200.0,14674.0,16946.0,16449.0,15421.0,14066.0,12911.0,12042.0,11882.0,11692.0
2,2nd Live-Birth,12192.0,12170.0,12485.0,14033.0,13370.0,13885.0,13937.0,14206.0,14913.0,...,13585.0,12631.0,11628.0,12431.0,12261.0,11438.0,10538.0,9824.0,9631.0,9620.0
3,3rd Live-Birth,3915.0,3982.0,4377.0,4669.0,4634.0,4721.0,4804.0,4721.0,4927.0,...,7083.0,6596.0,6624.0,7855.0,8318.0,7784.0,7256.0,6860.0,6800.0,6902.0
4,4th Live-Birth,1154.0,1180.0,1286.0,1373.0,1415.0,1381.0,1413.0,1380.0,1378.0,...,2648.0,2854.0,3497.0,4508.0,4821.0,4660.0,4588.0,4381.0,4728.0,5394.0


## 2. Clean Data

First we get rid of 3/4 birth order births - because they are not needed for our analysis.

In [109]:
clean_df = df.rename(columns={raw_df.columns[0]: 'Birth_Order'})

# filter for only 1st and 2nd births (our treatment and control groups)
clean_df = clean_df[clean_df['Birth_Order'].str.contains('1st|2nd', na=False, regex=True)].copy()

# clean up the Birth_Order labels
clean_df['Birth_Order'] = clean_df['Birth_Order'].str.strip()

clean_df

,Birth_Order,2024,2023,2022,2021,2020,2019,2018,2017,2016,...,1976,1975,1974,1973,1972,1971,1970,1969,1968,1967
1,1st Live-Birth,15769.0,15552.0,16739.0,17895.0,18414.0,18553.0,18211.0,18570.0,19392.0,...,17200.0,14674.0,16946.0,16449.0,15421.0,14066.0,12911.0,12042.0,11882.0,11692.0
2,2nd Live-Birth,12192.0,12170.0,12485.0,14033.0,13370.0,13885.0,13937.0,14206.0,14913.0,...,13585.0,12631.0,11628.0,12431.0,12261.0,11438.0,10538.0,9824.0,9631.0,9620.0


Next, we do a bunch of housekeeping. We melt the df to convert year columns into rows, make the year valus numeric, and the df by year.

In [110]:
# melt from wide to long format
df_long = clean_df.melt(
    id_vars=['Birth_Order'],
    var_name='Year',
    value_name='Births'
)
# convert to numeric
df_long['Year'] = pd.to_numeric(df_long['Year'], errors='coerce')

# Sort by Year and Birth_Order
df_long = df_long.sort_values(['Year', 'Birth_Order'])

df_long.head(10)

,Birth_Order,Year,Births
114,1st Live-Birth,1967,11692.0
115,2nd Live-Birth,1967,9620.0
112,1st Live-Birth,1968,11882.0
113,2nd Live-Birth,1968,9631.0
110,1st Live-Birth,1969,12042.0
111,2nd Live-Birth,1969,9824.0
108,1st Live-Birth,1970,12911.0
109,2nd Live-Birth,1970,10538.0
106,1st Live-Birth,1971,14066.0
107,2nd Live-Birth,1971,11438.0


## 3. The Parallel Trends Assumption

As mentioned in the introduction, it's important to ensure that treatment group and control group followed similar trends before the policy was enacted. This shows that without the treatment (Baby Bonus), both groups would have likely followed the exact trend in outcomes over time (1st/2nd births). This then means that any divergence we observe can be attributed to the policy itself. 

This is called the  **Parallel Trends Assumption**:

> *In the absence of the policy, the treatment and control groups would have followed the same trend over time.*

For this blog post, I'm going to test it visually, by plotting both groups over time and check if they move together *before* the intervention.


In [112]:
# creating the plot ()
fig = px.line(
    df_long[(df_long['Year'] >= 1990) & (df_long['Year'] <= 2010)],
    x='Year',
    y='Births',
    color='Birth_Order',
    title='1st vs 2nd Child Births in Singapore',
    labels={'Births': 'Number of Live Births', 'Birth_Order': 'Birth Order'},
    markers=True
)

# Add vertical line for the 2001 Baby Bonus (2nd child)
fig.add_vline(
    x=2001, 
    line_dash='dash', 
    line_color='green',
    annotation_text='Baby Bonus (2nd child)',
    annotation_position='top left'
)

# Add vertical line for the 2004 Baby Bonus (1st child)
fig.add_vline(
    x=2004, 
    line_dash='dash', 
    line_color='orange',
    annotation_text='Baby Bonus (1st child)',
    annotation_position='top right'
)

fig.show()

From a brief look at this, it does in fact seem as if the parralel trends requirement is met. Both graphs mostly have the same shape.

## 4. Carrying out the DiD

To figure out DiD using Python I took a look at this Kaggle notebook: https://www.kaggle.com/code/harrywang/difference-in-differences-in-python

Since I didn't want to use sklearn - which I have not used before, and has not been taught in class - to carry out the regression, I carried out a simplier pandas-only approach.

You can manually estimate a DiD by finding the mean outcome (births) for the treated group before and after the intervention, and similarly for the control group. Then you subtract the control group's change from the treatment group's change: 

> (Tpost - Tpre) - (Cpost - Cpre)

This effectively compares how much the treated group changed relative to the control group's trend.

The limitations of doing this manually using Pandas is that we will not be p-values (is the effect statistically significant), R-Squared, standard errors etc.

We'll focus on the window **1997-2003** to avoid confounding from:
- The 2004 extension of Baby Bonus to 1st children
- The 1987 "Have Three or More" policy reversal



In [113]:
# filter to our analysis window (1997-2003)
analysis_df = df_long[(df_long['Year'] >= 1997) & (df_long['Year'] <= 2003)].copy()

# create treatment variable (1 = 2nd child, 0 = 1st child)
analysis_df['Treatment'] = (analysis_df['Birth_Order'].str.contains('2nd')).astype(int)

# create post-policy variable (1 = 2001 onwards, 0 = before 2001)
analysis_df['Post'] = (analysis_df['Year'] >= 2001).astype(int)

# create interaction term (treatment x Post) - the group that recieved the baby bonus policy!
analysis_df['Treatment_x_Post'] = analysis_df['Treatment'] * analysis_df['Post']

analysis_df

,Birth_Order,Year,Births,Treatment,Post,Treatment_x_Post
54,1st Live-Birth,1997,19416.0,0,0,0
55,2nd Live-Birth,1997,17127.0,1,0,0
52,1st Live-Birth,1998,18860.0,0,0,0
53,2nd Live-Birth,1998,15030.0,1,0,0
50,1st Live-Birth,1999,18778.0,0,0,0
51,2nd Live-Birth,1999,15170.0,1,0,0
48,1st Live-Birth,2000,19930.0,0,0,0
49,2nd Live-Birth,2000,16948.0,1,0,0
46,1st Live-Birth,2001,17595.0,0,1,0
47,2nd Live-Birth,2001,14501.0,1,1,1


In [114]:
# calculate means for each group
did_table = analysis_df.groupby(['Treatment', 'Post'])['Births'].mean().unstack()
did_table

did_table.index = ['1st Child (Control)', '2nd Child (Treatment)']
did_table.columns = ['Pre-2001', 'Post-2001']

# calculate the difference for each group
did_table['Difference'] = did_table['Post-2001'] - did_table['Pre-2001']

did_table

,Pre-2001,Post-2001,Difference
1st Child (Control),19246.00,17175.666667,-2070.333333
2nd Child (Treatment),16068.75,14277.000000,-1791.750000


In [115]:

# Calculate DiD
did_estimate = (treatment_post - treatment_pre) - (control_post - control_pre)
did_estimate

np.float64(278.5833333333321)

To visualize this more clearly, we can index both groups to 2000 = 100. This makes it easier to compare relative changes between the two groups:

## 5. Conclusion

From the results above, we can see an estimate of about **279 babies** that could have been born because of the Baby Bonus Policy. But it's important to caveat this. Not only does this number seem quite low, as mentioned earlier, we don't know whether the effect is **statistically significant**. It seems quite likely that this policy did not in fact increase birth rates signficantly.

In addition whilst making this notebook, I noticed something interesting in the parallel trends graph. After the baby bonus was introduced for 1st borns, the number of births of first borns remained consistent, even increasing, whilst the second born births remained low and unchanging.

To visualise this better I did an indexed chart, having the year 2000 as 100%



In [116]:
# create indexed chart (2000 = 100)
idx_df = df_long[(df_long['Year'] >= 1990) & (df_long['Year'] <= 2010)].copy()
base = idx_df[idx_df['Year'] == 2000].set_index('Birth_Order')['Births']
idx_df['Index'] = idx_df.apply(lambda r: (r['Births'] / base[r['Birth_Order']]) * 100, axis=1)

fig2 = px.line(idx_df, x='Year', y='Index', color='Birth_Order',
               title='Indexed Birth Trends (2000 = 100)', markers=True)
fig2.add_vline(x=2003, line_dash='dash', line_color='green', annotation_text='Baby Bonus (1st)')
fig2.add_hline(y=100, line_dash='dot', line_color='gray')
fig2.show()

This could suggest a few things;

(1) The baby bonus policy might have been more effective for the first child as compared to the second. Maybe potential first-time parents are more 'price elastic' as compared to second-time parents. We can try and do a DiD for this, but the amounts for 1st and 2nd children are different.

(2) More damningly for me, it suggests that the population of treatment and control groups might not be the same. The concerns, wants, needs, demographics of parents having thier first vs second child might not be the same - though the parallel trends do seem to be mostly satisfied. 

On both of these implications, a more in-depth analysis could help. I.e. brodening the years to check for parallel trends, checking for statistical significance etc. 

Regardless, the conclusion still holds true: the **Baby Bonus policy (for second births) does not seem to have increased birth rates in Singapore significantly**. If it had done so, we would have seen a larger effect in both our plots and DiD estimate. 